# Memoria
Este código está diseñado para ilustrar y comparar tres funciones de memoria diferentes utilizadas en sistemas de IA o chatbots: **<span style="color:green">LastMemories</span>**, **<span style="color:green">LastTokens</span>** y **<span style="color:green">RelevantMemories</span>**. Cada una de estas funciones de memoria cumple un propósito distinto en la gestión y recuperación del historial de conversaciones.

## 1.- Setup inicial

### 1.1- Instalar librerías

In [1]:
#! pip install openai
#! pip install numpy
#! pip install tenacity
#! pip install python-dotenv

### 1.2.- Cargar librerías

In [2]:
import os
import json
import numpy
import openai
from datetime import datetime
from llm import generate_text, count_tokens
from dotenv import load_dotenv

### 1.3.- Variables de entorno

In [3]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

OpenAI API key: sk-eO...Re5lw
GPT-3.5-Turbo model: gpt-3.5-turbo
GPT-3.5-Turbo-16k model: gpt-3.5-turbo-16k
GPT-4 model: gpt-4


### 1.4.- Importar e instanciar las clases a comparar

In [4]:
from agents.memory.last_memories import LastMemories
from agents.memory.last_tokens import LastTokens
from agents.memory.relevant_memories import RelevantMemories

last_memories = LastMemories()
last_tokens = LastTokens()
relevant_memories = RelevantMemories()

## 2.- Importar conversación y almacenarla en las respectivas clases

In [5]:
with open("conversation.json", "r", encoding="utf-8") as json_file:
    loaded_file = json.load(json_file)
    conversation = loaded_file["conversation"]
for message in conversation:
    last_memories.add_to_memory(message["role"], message["content"])
    last_tokens.add_to_memory(message["role"], message["content"])
    relevant_memories.add_to_memory(message["role"], message["content"])

### Temas tratados en la conversación:

#### 1. Literatura 
   ###### - Autores
   ###### - Obras
   ###### - Géneros
#### 2. Deportes
   ###### - Fútbol
   ###### - Tenis
   ###### - Baloncesto
   ###### - Eventos deportivos
#### 4. Cine
   ###### - Películas (Clásicas)
   ###### - Directores
   ###### - Géneros
#### 5. Música
   ###### - Géneros
   ###### - Músicos
   ###### - Festivales
   ###### - Instrumentos musicales
   ###### - Canciones
#### 6. Lugares
   ###### - Viajes
   ###### - Monumentos
   ###### - Ciudades
   ###### - Lugares ficticios
   ###### - Estadios
#### 7. Libros
   ###### - Recomendaciones de libros
#### 8. Peliculas y Deportes
   ###### - Películas relacionadas con deportes
   ###### - Deportistas que incursionan en el cine
#### 9. Música y Cine
   ###### - Bandas sonoras
#### 10. Lugares y Cine
###### - Lugares turísticos es películas
#### 11. Lugares, Deportes y Películas
###### - Arena o estadios en películas

### Tokens 

In [6]:
tokens = 0
for message in conversation:
    tokens += count_tokens(message["content"])
print(f'Total de tokens de la conversación: {tokens}')

Total de tokens de la conversación: 8218


## 4.- Comparación de las funciones de Memoria

### 4.1.- Last Memories

In [7]:
memories = last_memories.get_last_memories()
for message in memories:
    print("\033[1m" + message["role"] + "\033[0m : " + message["content"])

assistant : Me alegra que hayas encontrado inspiración en nuestra conversación. La cultura es un reflejo de la diversidad y la creatividad humanas. Si alguna vez deseas continuar explorando estos temas o hablar sobre cualquier otro, estaré aquí para ti. ¡Hasta la próxima!
user : La conversación me ha recordado la importancia de la cultura en nuestras vidas y cómo la literatura, el cine, la música y los lugares pueden influir en nuestra percepción del mundo. Ha sido inspirador.
assistant : Totalmente de acuerdo. La música es una fuerza universal que puede enriquecer nuestras vidas de muchas maneras. ¿Hay alguna conclusión o reflexión final que quieras compartir después de nuestra conversación?
user : Definitivamente, la conexión entre la música y la vida es algo que me ha impactado. La música tiene el poder de influir en nuestras emociones y experiencias de maneras profundas.
assistant : Ha sido un placer conversar contigo sobre una variedad de temas, desde literatura y cine hasta lugar

### 4.2.- Last Tokens

In [10]:
tokens = last_tokens.get_last_tokens()
for message in tokens:
    print("\033[1m" + message["role"] + "\033[0m : " + message["content"])

assistant : Me alegra que hayas encontrado inspiración en nuestra conversación. La cultura es un reflejo de la diversidad y la creatividad humanas. Si alguna vez deseas continuar explorando estos temas o hablar sobre cualquier otro, estaré aquí para ti. ¡Hasta la próxima!
user : La conversación me ha recordado la importancia de la cultura en nuestras vidas y cómo la literatura, el cine, la música y los lugares pueden influir en nuestra percepción del mundo. Ha sido inspirador.
assistant : Totalmente de acuerdo. La música es una fuerza universal que puede enriquecer nuestras vidas de muchas maneras. ¿Hay alguna conclusión o reflexión final que quieras compartir después de nuestra conversación?
user : Definitivamente, la conexión entre la música y la vida es algo que me ha impactado. La música tiene el poder de influir en nuestras emociones y experiencias de maneras profundas.
assistant : Ha sido un placer conversar contigo sobre una variedad de temas, desde literatura y cine hasta lugar

### 4.3.- Relevant Memories

In [12]:
relevant = relevant_memories.get_relevant_memories("película")
for message in relevant:
    print("\033[1m" + message["role"] + "\033[0m : " + message["content"])

assistant : Películas como 'La vida de Pi' o 'El Renacido' muestran la majestuosidad y la crueldad de la naturaleza en su máxima expresión. La cinematografía en estas películas es impresionante.
assistant : ¡Claro! 'Carros de Fuego' es una película icónica sobre atletas olímpicos y su lucha por el éxito. La música y las actuaciones hacen que sea una experiencia memorable.
user : Esa es una elección clásica. Cambiando de tema, ¿tienes alguna experiencia en el cine que recuerdes con cariño?
assistant : Aunque no tengo experiencias personales, he aprendido de muchas interacciones de personas en el cine. Las historias compartidas por los cinéfilos son valiosas y reflejan su pasión por el séptimo arte.
user : El cine es una forma poderosa de conectar a las personas. ¿Tienes alguna película favorita que haya influido en tu comprensión del mundo?
assistant : No tengo preferencias, pero películas como 'El Club de la Pelea' o 'Matrix' exploran conceptos filosóficos y la percepción de la realida